# FindCare — HTML-Fragment API Backend (Gradio + FastAPI)

**File:** FindCare_HTML_Fragment_Backend.ipynb  
**Author:** Skip Snow  
**Co-Author:** GPT-5  
**Copyright:** Copyright (c) 2025 Skip Snow. All rights reserved.

This notebook implements the **/ui/** endpoints discussed, where **every server call returns HTML fragments** that the browser renders into the wireframe regions:

- header_left
- header_nav
- left_rail
- results_box
- content_pane
- prompt_box
- toast


In [ ]:
# If needed, install dependencies (uncomment in a fresh environment)
# !pip -q install gradio fastapi uvicorn python-multipart pydantic

import gradio as gr
from fastapi import UploadFile, File, Form, Body
from fastapi.responses import JSONResponse, HTMLResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional, Dict, Any
import time
import uuid
import re
import random
import threading
import webbrowser


## 1) Session store

Simple in-memory session. Frontend should pass `sessionId` on each call (header or JSON field). If not provided, we create one.

In [ ]:
class SessionStore:
    def __init__(self):
        self._sessions: Dict[str, Dict[str, Any]] = {}

    def get_or_create(self, session_id: Optional[str]) -> str:
        sid = session_id or str(uuid.uuid4())
        if sid not in self._sessions:
            self._sessions[sid] = {
                "created_at": time.time(),
                "prompts": [],
                "results": [],
                "copies": [],
                "deletes": [],
                "selected_state": None,
                "model": "mock-default",
                "active_nav": "recipe-metrics",  # recipe-metrics | about | contact | privacy
                "toast": ""
            }
        return sid

    def s(self, sid: str) -> Dict[str, Any]:
        return self._sessions[sid]

STORE = SessionStore()


## 2) Mock data + helpers

Replace with Mongo/provider directory + specialty metadata + vector search later.

In [ ]:
STATE_ABBR = {
    "Alabama":"AL","Alaska":"AK","Arizona":"AZ","Arkansas":"AR","California":"CA","Colorado":"CO","Connecticut":"CT",
    "Delaware":"DE","Florida":"FL","Georgia":"GA","Hawaii":"HI","Idaho":"ID","Illinois":"IL","Indiana":"IN","Iowa":"IA",
    "Kansas":"KS","Kentucky":"KY","Louisiana":"LA","Maine":"ME","Maryland":"MD","Massachusetts":"MA","Michigan":"MI",
    "Minnesota":"MN","Mississippi":"MS","Missouri":"MO","Montana":"MT","Nebraska":"NE","Nevada":"NV","New Hampshire":"NH",
    "New Jersey":"NJ","New York":"NY","North Carolina":"NC","North Dakota":"ND","Ohio":"OH","Oklahoma":"OK","Oregon":"OR",
    "Pennsylvania":"PA","Rhode Island":"RI","South Carolina":"SC","South Dakota":"SD","Tennessee":"TN","Texas":"TX","Utah":"UT",
    "Vermont":"VT","Virginia":"VA","Washington":"WA","West Virginia":"WV","Wisconsin":"WI","Wyoming":"WY"
}
VALID_STATE_ABBR = set(STATE_ABBR.values())

TYPE_COLOR = {
    "Hospital": "#3b82f6",
    "Clinic": "#10b981",
    "Urgent Care": "#f59e0b",
    "Private Practice": "#8b5cf6",
    "Imaging Center": "#ef4444",
}

MOCK_PROVIDERS = [
    {"name":"General Hospital", "type":"Hospital", "states":["CA","TX","FL","NY"], "distance":"2.3 miles"},
    {"name":"Family Care Clinic", "type":"Clinic", "states":["CA","WA","OR","NV"], "distance":"1.8 miles"},
    {"name":"Urgent Care Center", "type":"Urgent Care", "states":["TX","LA","OK","AR"], "distance":"3.5 miles"},
    {"name":"Heart & Vascular Associates", "type":"Private Practice", "states":["CA","NY","PA","MA"], "distance":"4.1 miles"},
    {"name":"Advanced Imaging", "type":"Imaging Center", "states":["FL","GA","NC","SC"], "distance":"2.9 miles"},
]

def normalize_state(state: str) -> str:
    state = (state or "").strip()
    if not state:
        return ""
    if len(state) == 2 and state.isalpha():
        abbr = state.upper()
        return abbr
    for name, abbr in STATE_ABBR.items():
        if name.lower() == state.lower():
            return abbr
    return state[:2].upper()

def infer_state_from_text(prompt: str) -> Optional[str]:
    m = re.search(r"\b([A-Z]{2})\b", prompt or "")
    if m:
        abbr = m.group(1)
        if abbr in VALID_STATE_ABBR:
            return abbr
    lowered = (prompt or "").lower()
    for name, abbr in STATE_ABBR.items():
        if name.lower() in lowered:
            return abbr
    return None

def mock_search(prompt: str, selected_state: Optional[str]=None) -> List[dict]:
    state = selected_state or infer_state_from_text(prompt)
    needle = (prompt or "").lower().strip()

    results = []
    base_id = int(time.time() * 1000)

    for i, p in enumerate(MOCK_PROVIDERS):
        if state and state not in p["states"]:
            continue

        if needle:
            if ("cardio" in needle) and ("Heart" not in p["name"]):
                continue
            if ("urgent" in needle) and (p["type"] != "Urgent Care"):
                continue
            if ("imaging" in needle or "xray" in needle or "mri" in needle) and (p["type"] != "Imaging Center"):
                continue

        results.append({
            "id": base_id + i,
            "name": p["name"],
            "type": p["type"],
            "color": TYPE_COLOR.get(p["type"], "#6b7280"),
            "distance": p["distance"],
        })

    if not results:
        results = [{
            "id": base_id,
            "name": "No matching providers (mock)",
            "type": "Clinic",
            "color": TYPE_COLOR["Clinic"],
            "distance": "N/A",
        }]
    return results

async def summarize_uploads(files: Optional[List[UploadFile]]) -> str:
    if not files:
        return ""
    parts = []
    for f in files:
        try:
            raw = await f.read()
            parts.append(f"{f.filename} ({len(raw)} bytes)")
        except Exception:
            parts.append(f"{getattr(f,'filename','(unknown)')} (unreadable)")
    return " | ".join(parts)


## 3) HTML fragment renderers

These produce the HTML that the client inserts into each region. Adjust styling to match your tone later.

In [ ]:
def render_header_left() -> str:
    return "<div class='fc-header-left'><strong>Logo</strong></div>"

def render_header_nav(active: str) -> str:
    tabs = [
        ("recipe-metrics", "Tool’s recipe &amp; metrics"),
        ("about", "About ‘Find care’"),
        ("contact", "Contact Skip"),
        ("privacy", "Privacy Policy"),
    ]
    items = []
    for key, label in tabs:
        cls = "fc-nav-item active" if key == active else "fc-nav-item"
        items.append(f"<button class='{cls}' data-nav='{key}'>{label}</button>")
    return "<div class='fc-header-nav'>" + "".join(items) + "</div>"

def render_left_rail(model_id: str) -> str:
    return f"""
<div class='fc-left-rail'>
  <div class='fc-left-rail-note'>JS will maintain a summary of the session. This is scoreable.</div>
  <div class='fc-left-rail-buttons'>
    <button class='fc-btn' data-action='insurance'>Insurance portal</button>
    <button class='fc-btn' data-action='emr'>EMR</button>
    <button class='fc-btn' data-action='transcript'>Get transcript &amp; summary</button>
    <button class='fc-btn' data-action='models'>Choose Models</button>
  </div>
  <div class='fc-left-rail-model'>Model: <code>{model_id}</code></div>
</div>
"""

def render_results_box(results: List[dict]) -> str:
    if not results:
        return "<div class='fc-results-empty'>No results yet.</div>"
    rows = []
    for r in results:
        rid = r.get("id")
        color = r.get("color", "#6b7280")
        name = r.get("name", "")
        typ = r.get("type", "")
        dist = r.get("distance", "")
        rows.append(f"""
<div class='fc-result-row' data-id='{rid}' style='border-left:6px solid {color}'>
  <div class='fc-result-title'>{name}</div>
  <div class='fc-result-meta'>{typ} • {dist}</div>
  <div class='fc-result-actions'>
    <button class='fc-icon' title='Copy' data-action='copy' data-id='{rid}'>📄</button>
    <button class='fc-icon' title='Delete' data-action='delete' data-id='{rid}'>🧽</button>
    <button class='fc-icon' title='Open' data-action='link' data-id='{rid}'>🔗</button>
  </div>
</div>
""")
    return "<div class='fc-results-box'>" + "".join(rows) + "</div>"

def render_map(selected_state: Optional[str], mode: str = "splash") -> str:
    s = selected_state or "United States"
    return f"""
<div class='fc-map'>
  <div class='fc-map-title'>Map</div>
  <div class='fc-map-placeholder'>
    <div><strong>API PENDING:</strong> Interactive US map rendering.</div>
    <div>Selected: <code>{s}</code></div>
    <div>Mode: <code>{mode}</code></div>
    <div class='fc-map-hint'>Client should call <code>/ui/map/hover</code> and <code>/ui/map/select</code>.</div>
  </div>
</div>
"""

def render_content_pane(session: Dict[str, Any], view: str, payload: Optional[Dict[str, Any]] = None) -> str:
    selected_state = session.get("selected_state")
    if view == "splash":
        return "<div class='fc-content-pane'>" + render_map(selected_state, "splash") + "</div>"

    if view == "nav":
        page = (payload or {}).get("page", "recipe-metrics")
        if page == "recipe-metrics":
            return "<div class='fc-content-pane'><h2>Tool’s recipe &amp; metrics</h2><p><strong>API PENDING:</strong> Define metrics (latency, hits, costs, model usage, etc.).</p></div>"
        if page == "about":
            return "<div class='fc-content-pane'><h2>About ‘Find care’</h2><p><strong>API PENDING:</strong> Provide About copy.</p></div>"
        if page == "contact":
            return "<div class='fc-content-pane'><h2>Contact Skip</h2><p><strong>API PENDING:</strong> Provide contact content / form behavior.</p></div>"
        if page == "privacy":
            return "<div class='fc-content-pane'><h2>Privacy Policy</h2><p><strong>API PENDING:</strong> Provide privacy policy content.</p></div>"
        return f"<div class='fc-content-pane'><p>Unknown page: {page}</p></div>"

    if view == "map-hover":
        st = (payload or {}).get("state") or selected_state or ""
        return f"<div class='fc-content-pane'>{render_map(st, 'hover')}<div class='fc-hover-panel'><h3>Hover: {st}</h3><p><strong>API PENDING:</strong> State hover text source &amp; content.</p></div></div>"

    if view == "map-select":
        st = (payload or {}).get("state") or selected_state or ""
        return f"<div class='fc-content-pane'>{render_map(st, 'select')}<div class='fc-state-panel'><h3>Selected: {st}</h3><p><strong>API PENDING:</strong> State detail content (coverage, regulations, etc.).</p></div></div>"

    if view == "provider-detail":
        rid = (payload or {}).get("resultId")
        return f"<div class='fc-content-pane'><h2>Provider Detail</h2><p><strong>API PENDING:</strong> provider detail view for <code>{rid}</code>.</p><p>Option: render full profile here OR return URL to open in new tab.</p></div>"

    if view == "insurance":
        return "<div class='fc-content-pane'><h2>Insurance Portal</h2><p><strong>API PENDING:</strong> Decide: return redirect URL vs render portal instructions here.</p></div>"

    if view == "emr":
        return "<div class='fc-content-pane'><h2>EMR</h2><p><strong>API PENDING:</strong> Decide: return redirect URL vs render EMR access instructions here.</p></div>"

    if view == "transcript":
        transcript = (payload or {}).get("transcript", "")
        summary = (payload or {}).get("summary", "")
        return f"<div class='fc-content-pane'><h2>Transcript &amp; Summary</h2><h3>Summary</h3><pre>{summary}</pre><h3>Transcript</h3><pre>{transcript}</pre></div>"

    if view == "models":
        current = session.get("model")
        return f"""<div class='fc-content-pane'><h2>Choose Models</h2>
<p>Current: <code>{current}</code></p>
<ul>
  <li><button class='fc-btn' data-model='mock-default'>Mock Default</button></li>
  <li><button class='fc-btn' data-model='openai-gpt'>OpenAI (future)</button> <em>(API PENDING: integration)</em></li>
  <li><button class='fc-btn' data-model='local-llama'>Local Llama (future)</button> <em>(API PENDING: integration)</em></li>
</ul></div>"""

    if view == "search":
        prompt = (payload or {}).get("prompt", "")
        file_summary = (payload or {}).get("files", "")
        return f"<div class='fc-content-pane'><h2>Search</h2><p><strong>Prompt:</strong> {prompt}</p><p><strong>Files:</strong> {file_summary or '(none)'}</p><p><strong>API PENDING:</strong> Show richer narrative, filters, or map overlays for results.</p></div>"

    return "<div class='fc-content-pane'><p>Unknown view.</p></div>"

def render_prompt_box() -> str:
    return """
<div class='fc-prompt-box'>
  <div class='fc-prompt-title'>Prompt Box</div>
  <textarea id='fc-prompt' rows='4' placeholder='Type your request...'></textarea>
  <div class='fc-prompt-actions'>
    <button class='fc-btn' data-action='submit-prompt'>Submit</button>
  </div>
</div>
"""

def render_toast(session: Dict[str, Any]) -> str:
    t = session.get("toast") or ""
    if not t:
        return ""
    return f"<div class='fc-toast'>{t}</div>"

def response_envelope(session_id: str, session: Dict[str, Any], event: str, data: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    html = {
        "header_left": render_header_left(),
        "header_nav": render_header_nav(session.get("active_nav", "recipe-metrics")),
        "left_rail": render_left_rail(session.get("model", "mock-default")),
        "results_box": render_results_box(session.get("results", [])),
        "content_pane": render_content_pane(session, "splash"),
        "prompt_box": render_prompt_box(),
        "toast": render_toast(session),
    }
    if data and "content_pane" in data:
        html["content_pane"] = data["content_pane"]
    if data and "results_box" in data:
        html["results_box"] = data["results_box"]
    if data and "header_nav" in data:
        html["header_nav"] = data["header_nav"]
    if data and "left_rail" in data:
        html["left_rail"] = data["left_rail"]
    if data and "toast" in data:
        html["toast"] = data["toast"]

    return {
        "ok": True,
        "event": event,
        "html": html,
        "state": {
            "sessionId": session_id,
            "selectedState": session.get("selected_state"),
            "modelId": session.get("model"),
            "activeNav": session.get("active_nav"),
        },
        "data": (data or {})
    }


## 4) API payload models

In [ ]:
class SessionPayload(BaseModel):
    sessionId: Optional[str] = None

class ResultPayload(SessionPayload):
    resultId: int

class MapPayload(SessionPayload):
    state: str

class ModelSelectPayload(SessionPayload):
    modelId: str


## 5) Gradio app + FastAPI routes (/ui/*)

All routes return HTML fragments in a standard envelope.

In [ ]:
with gr.Blocks(title="FindCare Backend (HTML Fragments)") as demo:
    gr.Markdown("""## FindCare Backend (HTML Fragment API)
This Gradio UI is optional; your real client is the browser UI.
Use this panel to keep the FastAPI server running in notebooks.""")

app = demo.app  # FastAPI app

# CORS: allow your React dev server + local variants. Tighten for production.
app.add_middleware(
    CORSMiddleware,
    allow_origins=[
        "http://localhost:5173",
        "http://127.0.0.1:5173",
        "http://localhost:3000",
        "http://127.0.0.1:3000",
    ],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


In [ ]:
# ----------------------------
# FindCare API (Spec-compliant)
# - First page is served immediately at GET /
# - Payloads are JSON; no prose/motivational copy
# - Endpoints match FRONTEND_API_SPECIFICATION.md
# ----------------------------

# Wireframe-like first page (austere). This is the real browser UI.
FIRST_PAGE_HTML = r"""<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>FindCare</title>
  <style>
    :root{
      --blueLeft1:#0f5f7a;
      --blueLeft2:#0b3f52;
      --blueTop:#5aa4d6;
      --leftTop:#5db2d7;
      --logoBg:#9ac3dd;
      --tan:#caa77a;
      --pink:#efd6c9;
      --border:#1b1b1b;
      --text:#111;
    }
    html,body{height:100%;margin:0;font-family:Segoe UI, Arial, sans-serif;color:var(--text);background:#f6f6f6;}
    .frame{
      width: 96vw;
      height: 92vh;
      margin: 2vh auto;
      border: 2px solid var(--border);
      background: white;
      display: grid;
      grid-template-columns: 150px 1fr;
      grid-template-rows: 54px 1fr 170px;
    }

    .left{
      grid-row: 1 / span 3;
      grid-column: 1;
      border-right: 2px solid var(--border);
      background: linear-gradient(var(--blueLeft1), var(--blueLeft2));
      display: grid;
      grid-template-rows: 54px 1fr 1fr;
      min-width: 150px;
    }
    .logo{
      border-bottom: 2px solid var(--border);
      background: var(--logoBg);
      display:flex;
      align-items:center;
      justify-content:center;
      font-weight:700;
    }
    .leftTop{
      padding: 10px;
      border-bottom: 2px solid var(--border);
      background: var(--leftTop);
      font-size: 12px;
      line-height: 1.2;
      overflow:auto;
    }
    .leftButtons{
      padding: 10px;
      display:flex;
      flex-direction:column;
      gap:10px;
    }
    .btn{
      border: 2px solid var(--border);
      background: #7ed074;
      padding: 10px 8px;
      font-size: 12px;
      text-align:left;
      cursor:pointer;
      user-select:none;
    }
    .btn:nth-child(2){background:#66c85b;}
    .btn:nth-child(3){background:#66b8ff;}
    .btn:nth-child(4){background:#63d2a6;}

    .topnav{
      grid-column: 2;
      grid-row: 1;
      border-bottom: 2px solid var(--border);
      background: var(--blueTop);
      display:grid;
      grid-template-columns: 1fr 1fr 1fr 1fr;
      align-items:stretch;
      position:relative;
    }
    .tab{
      border-left: 2px solid var(--border);
      display:flex;
      align-items:center;
      justify-content:center;
      font-weight:600;
      font-size: 13px;
      cursor:pointer;
    }
    .tab:first-child{border-left:none;}
    .tone{
      position:absolute;
      left: 50%;
      transform: translateX(-50%);
      top: -18px;
      font-style: italic;
      font-weight: 600;
      font-size: 12px;
      opacity: 0.85;
      color: #1b1b1b;
    }

    .middle{
      grid-column: 2;
      grid-row: 2;
      display:grid;
      grid-template-columns: 360px 1fr;
      border-bottom: 2px solid var(--border);
      background: var(--pink);
    }
    .resultBox{
      border-right: 2px solid var(--border);
      background: var(--tan);
      padding: 10px 12px;
      overflow:auto;
    }
    .resultTitle{font-weight:700;text-align:center;margin:2px 0 10px;}
    .resultHint{font-size:12px;line-height:1.25;margin-bottom:12px;}
    .resultList{display:flex;flex-direction:column;gap:8px;}
    .resultItem{
      border: 2px solid var(--border);
      background: rgba(255,255,255,0.55);
      padding: 8px;
      display:grid;
      grid-template-columns: 1fr auto;
      gap: 8px;
      align-items:center;
    }
    .icons{display:flex;gap:6px;}
    .iconBtn{
      width: 22px;height: 22px;
      border: 2px solid var(--border);
      background:#fff;
      display:flex;align-items:center;justify-content:center;
      font-size: 12px;
      cursor:pointer;
    }

    .contentPane{padding:10px 12px; overflow:auto;}
    .contentTitle{
      font-size: 28px;
      font-family: Georgia, "Times New Roman", serif;
      font-weight: 500;
      text-align:center;
      margin: 8px 0 12px;
    }
    .contentSubtitle{ text-align:center; font-size:12px; font-style:italic; margin-bottom: 8px; }
    .mapBox{
      margin: 10px auto 0;
      width: 86%;
      height: 360px;
      border: 2px solid var(--border);
      background: rgba(255,255,255,0.35);
      display:flex;
      align-items:center;
      justify-content:center;
      font-size: 12px;
      text-align:center;
      padding: 12px;
    }

    .prompt{
      grid-column: 2;
      grid-row: 3;
      background: var(--pink);
      display:flex;
      align-items:center;
      justify-content:center;
      position:relative;
    }
    .promptTitle{
      position:absolute;
      top: 12px;
      font-size: 22px;
      font-family: Georgia, "Times New Roman", serif;
    }
    .promptInner{
      width: 92%;
      height: 62%;
      display:grid;
      grid-template-columns: 1fr 120px;
      gap: 10px;
      align-items:stretch;
      margin-top: 34px;
    }
    textarea{
      border: 2px solid var(--border);
      resize:none;
      padding: 10px;
      font-size: 14px;
      outline:none;
    }
    .send{
      border: 2px solid var(--border);
      background:#fff;
      cursor:pointer;
      font-weight:700;
    }
  </style>
</head>
<body>
  <div class="frame">
    <aside class="left">
      <div class="logo">Logo</div>

      <div class="leftTop">
        <div style="font-weight:700;margin-bottom:6px;">Session Summary</div>
        <div id="sessionSummary" style="white-space:pre-wrap;"></div>
      </div>

      <div class="leftButtons">
        <div class="btn" data-action="insurance" title="Insurance Portal">Insurance portal Button</div>
        <div class="btn" data-action="emr" title="EMR Access">EMR button</div>
        <div class="btn" data-action="transcript" title="Transcript & Summary">Get transcript and summary button</div>
        <div class="btn" data-action="models" title="Choose Models">Choose Models button</div>
      </div>
    </aside>

    <header class="topnav">
      <div class="tone">Web site's tone is austere.</div>
      <div class="tab" data-page="home">Tool’s recipe &amp; metrics</div>
      <div class="tab" data-page="about">About ‘find care’</div>
      <div class="tab" data-page="contact">Contact Skip</div>
      <div class="tab" data-page="support">Privacy Policy</div>
    </header>

    <main class="middle">
      <section class="resultBox">
        <div class="resultTitle">Result box</div>
        <div class="resultHint">
          Results support: copy, delete, link. Hover text must say “copy” and “delete”.<br/><br/>
          Result box is vertically scrollable.
        </div>
        <div class="resultList" id="resultList"></div>
      </section>

      <section class="contentPane">
        <div class="contentTitle">Content Pane</div>
        <div class="contentSubtitle">Map on loaded splash page</div>
        <div class="mapBox" id="mapBox">
          US map placeholder. Click actions should call <code>/api/state</code>.
        </div>
        <div style="margin-top:14px;font-size:12px;text-align:center;opacity:.85;">
          Current Content scrollable vertical and horizontal
        </div>
      </section>
    </main>

    <section class="prompt">
      <div class="promptTitle">Prompt Box</div>
      <div class="promptInner">
        <textarea id="prompt" placeholder="Ask within domain… (specialty, location, insurance, provider type)"></textarea>
        <button class="send" id="sendBtn">SEND</button>
      </div>
    </section>
  </div>

  <script>
    const sessionSummary = [];
    function renderSessionSummary(){
      const el = document.getElementById("sessionSummary");
      const last5 = sessionSummary.slice(-5);
      el.textContent = last5.map((q,i)=>`${i+1}. ${q}`).join("\\n");
    }
    function escapeHtml(s){
      return (s ?? "").replace(/[&<>"']/g, (c)=>({ "&":"&amp;","<":"&lt;",">":"&gt;",'"':"&quot;","'":"&#39;" }[c]));
    }
    function addResultItem(r){
      const list = document.getElementById("resultList");
      const item = document.createElement("div");
      item.className = "resultItem";
      const color = r.color || "#999999";
      item.innerHTML = `
        <div>
          <div style="display:flex;align-items:center;gap:8px;">
            <div style="width:10px;height:10px;border-radius:50%;background:${color};border:1px solid #111;"></div>
            <div style="font-weight:700;">${escapeHtml(r.name || "")}</div>
          </div>
          <div style="font-size:12px;opacity:.9;margin-top:4px;">
            ${escapeHtml(r.type || "")} • ${escapeHtml(r.distance || "")}
          </div>
        </div>
        <div class="icons">
          <div class="iconBtn" title="copy" data-action="copy">⧉</div>
          <div class="iconBtn" title="delete" data-action="delete">⌫</div>
          <div class="iconBtn" title="open" data-action="open">↗</div>
        </div>
      `;
      list.prepend(item);

      item.addEventListener("click", async (e)=>{
        const btn = e.target.closest(".iconBtn");
        if(!btn) return;
        const act = btn.getAttribute("data-action");

        if(act==="copy"){
          await navigator.clipboard.writeText(JSON.stringify(r, null, 2));
          fetch("/api/copy", {method:"POST", headers:{"Content-Type":"application/json"}, body: JSON.stringify({resultId: r.id})});
        }
        if(act==="delete"){
          fetch("/api/result", {method:"DELETE", headers:{"Content-Type":"application/json"}, body: JSON.stringify({resultId: r.id})});
          item.remove();
        }
        if(act==="open"){
          const resp = await fetch("/api/link", {method:"POST", headers:{"Content-Type":"application/json"}, body: JSON.stringify({resultId: r.id})});
          const data = await resp.json();
          if(data.url){ window.open(data.url, "_blank"); }
        }
      });
    }

    async function doSearch(promptText){
      const fd = new FormData();
      fd.append("prompt", promptText);
      const resp = await fetch("/api/search", { method:"POST", body: fd });
      const data = await resp.json();
      (data.results || []).forEach(addResultItem);
    }

    document.getElementById("sendBtn").addEventListener("click", async ()=>{
      const promptEl = document.getElementById("prompt");
      const q = (promptEl.value || "").trim();
      if(!q) return;
      sessionSummary.push(q);
      renderSessionSummary();
      await doSearch(q);
      promptEl.value = "";
    });

    document.querySelectorAll(".btn").forEach(btn=>{
      btn.addEventListener("click", async ()=>{
        const action = btn.getAttribute("data-action");
        const resp = await fetch("/api/action", {method:"POST", headers:{"Content-Type":"application/json"}, body: JSON.stringify({action})});
        const data = await resp.json();
        if(data && (data.transcript || data.summary)){
          document.getElementById("mapBox").innerHTML =
            `<div style="text-align:left;width:100%;height:100%;overflow:auto;">
               <div style="font-weight:700;margin-bottom:8px;">Transcript</div>
               <pre style="white-space:pre-wrap;">${escapeHtml(data.transcript || "")}</pre>
               <div style="font-weight:700;margin:12px 0 8px;">Summary</div>
               <pre style="white-space:pre-wrap;">${escapeHtml(data.summary || "")}</pre>
             </div>`;
        }
      });
    });

    document.querySelectorAll(".tab").forEach(tab=>{
      tab.addEventListener("click", async ()=>{
        const page = tab.getAttribute("data-page");
        const resp = await fetch("/api/navigation", {method:"POST", headers:{"Content-Type":"application/json"}, body: JSON.stringify({page})});
        const data = await resp.json();
        if(data && data.contentHtml){
          document.getElementById("mapBox").innerHTML = data.contentHtml;
        }
      });
    });

    renderSessionSummary();
  </script>
</body>
</html>
"""

# Session state (prototype)
_API_SESSION: Dict[str, Any] = {"transcript": [], "results": {}}

def _api_mock_result() -> Dict[str, Any]:
    rid = int(time.time() * 1000) + random.randint(0, 999)
    r = {
        "id": rid,
        "name": f"Provider {rid % 1000}",
        "type": random.choice(["Hospital", "Clinic", "Urgent Care", "Private Practice"]),
        "color": random.choice(["#3b82f6", "#10b981", "#f59e0b", "#ef4444"]),
        "distance": f"{round(random.uniform(0.5, 12.0), 1)} miles",
    }
    _API_SESSION["results"][rid] = r
    return r

# Request bodies
class _DeleteBody(BaseModel):
    resultId: int

class _CopyBody(BaseModel):
    resultId: int

class _ActionBody(BaseModel):
    action: str

class _StateBody(BaseModel):
    state: str
    action: Optional[str] = "select"

class _NavBody(BaseModel):
    page: str

class _LinkBody(BaseModel):
    resultId: int

# Serve first page immediately (no splash)
@app.get("/", response_class=HTMLResponse)
def _home():
    return HTMLResponse(FIRST_PAGE_HTML)

# Spec endpoints

@app.post("/api/search")
async def api_search(prompt: str = Form(...), files: Optional[List[UploadFile]] = File(None)):
    _API_SESSION["transcript"].append(f"User: {prompt}")
    results = [_api_mock_result() for _ in range(3)]
    return JSONResponse({"results": results})

@app.delete("/api/result")
def api_delete_result(body: _DeleteBody = Body(...)):
    _API_SESSION["results"].pop(body.resultId, None)
    return JSONResponse({"ok": True})

@app.post("/api/copy")
def api_copy(body: _CopyBody):
    return JSONResponse({"ok": True})

@app.post("/api/action")
def api_action(body: _ActionBody):
    action = body.action.lower().strip()
    if action == "insurance":
        return JSONResponse({"redirectUrl": "/insurance-portal"})
    if action == "emr":
        return JSONResponse({"redirectUrl": "/emr-access"})
    if action == "transcript":
        transcript = "\n".join(_API_SESSION["transcript"])
        summary = "Session summary: " + (_API_SESSION["transcript"][-1] if _API_SESSION["transcript"] else "No activity.")
        return JSONResponse({"transcript": transcript, "summary": summary, "downloadUrl": None})
    if action == "models":
        return JSONResponse({
            "availableModels": [
                {"id": "gpt-5", "name": "GPT-5", "description": "General reasoning + tool use"},
                {"id": "gpt-5-mini", "name": "GPT-5 Mini", "description": "Lower latency / cost"},
            ],
            "currentModel": "gpt-5"
        })
    return JSONResponse({"error": "Unknown action", "code": "UNKNOWN_ACTION"}, status_code=400)

@app.post("/api/state")
def api_state(body: _StateBody):
    providers = [_api_mock_result() for _ in range(5)]
    return JSONResponse({
        "providers": providers,
        "stateInfo": {"name": body.state, "insuranceCoverage": "TBD", "regulations": "TBD"}
    })

@app.post("/api/navigation")
def api_navigation(body: _NavBody):
    page = body.page.lower().strip()
    content = {
        "home": "<div style='text-align:left;'>Recipe &amp; Metrics (placeholder)</div>",
        "about": "<div style='text-align:left;'>About FindCare (placeholder)</div>",
        "services": "<div style='text-align:left;'>Services (placeholder)</div>",
        "contact": "<div style='text-align:left;'>Contact (placeholder)</div>",
        "support": "<div style='text-align:left;'>Privacy Policy (placeholder)</div>",
    }.get(page, "<div style='text-align:left;'>Unknown page</div>")
    return JSONResponse({"contentHtml": content})

# Compatibility endpoints from COMPLETE_CODE_EXPORT.md
@app.post("/api/query")
async def api_query(prompt: str = Form(...), files: Optional[List[UploadFile]] = File(None)):
    return await api_search(prompt=prompt, files=files)

@app.post("/api/link")
def api_link(body: _LinkBody):
    return JSONResponse({"url": f"/provider/{body.resultId}"})


# Browser opener for notebooks (Gradio may not open reliably)
def _open_browser_after_start(host: str, port: int, path: str = "/"):
    url = f"http://{host}:{port}{path}"
    def _open():
        time.sleep(0.8)
        webbrowser.open(url, new=2)
    threading.Thread(target=_open, daemon=True).start()

## 6) Launch

Run this cell to start the server inside Jupyter.

In [ ]:
HOST = "127.0.0.1"
PORT = 7871

_open_browser_after_start(HOST, PORT, "/")

demo.queue().launch(
    server_name=HOST,
    server_port=PORT,
    show_api=False,
    prevent_thread_lock=True,
    inbrowser=False
)
